# Summarise index details

This notebook counts the number of rows in each index and calculates the total for the whole repository. It formats the results in nice HTML and Markdown tables for easy browsing.

In [2]:
from urllib.parse import urljoin

import pandas as pd
from IPython.display import HTML, display
from tabulate import tabulate

## Add links and totals to the list of indexes

In [10]:
# Load the index data
df = pd.read_csv("indexes.csv").sort_values(by="title")

In [11]:
def make_download_link(url):
    """
    Create a link to download the CSV file from GitHub
    """
    slug = url.strip("/").split("/")[-1]
    filename = f"{slug}.csv"
    url = urljoin(
        "https://media.githubusercontent.com/media/wragge/srnsw-indexes/master/data/",
        filename,
    )
    link = '<a href="{}">CSV file</a>'.format(url)
    return link


# Create a HTML link to the index data on the NSWSA site
df["web"] = df["url"].apply(lambda x: '<a href="{}">Browse index</a>'.format(x))

# Create a HTML link to download the CSV file from GitHub
df["download"] = df["url"].apply(lambda x: make_download_link(x))

In [12]:
def count_rows(url):
    """
    Count the number of rows in a CSV file.
    """
    slug = url.strip("/").split("/")[-1]
    url = urljoin(
        "https://media.githubusercontent.com/media/wragge/srnsw-indexes/master/data/",
        f"{slug}.csv",
    )
    df = pd.read_csv(url, dtype=object)
    return df.shape[0]


# Add number of rows in the CSV
df["rows"] = df["url"].apply(lambda x: count_rows(x))

In [13]:
# How many rows in the whole repository?
df["rows"].sum()

2481881

In [14]:
# Which index has the most number of rows?
df.loc[df["rows"].idxmax()]

title                           Deceased estates index 1880-1958
url            https://mhnsw.au/indexes/deceased-estates/dece...
description    Researching deceased estates files is a comple...
category                                        Deceased estates
web            <a href="https://mhnsw.au/indexes/deceased-est...
download       <a href="https://media.githubusercontent.com/m...
rows                                                      577891
Name: 26, dtype: object

## Summarise the results of the harvest

In [15]:
"Currently: {} indexes harvested with {:,} rows of data.".format(
    df.shape[0], df["rows"].sum()
)

'Currently: 75 indexes harvested with 2,481,881 rows of data.'

Make a nicely formatted table in both HTML and Markdown.

In [16]:
# Select the columns that we want
columns = df[["title", "rows", "download", "web"]]

# Create a list of headers
headers = ["Title", "Number of rows", "Download data", "View at State Archives"]

# Use Tabulate to generate a HTML table
display(
    HTML(
        tabulate(
            columns, headers=headers, showindex=False, tablefmt="unsafehtml", intfmt=","
        )
    )
)

# Write a GitHub Markdown formatted version of the table to a file
with open("indexes.md", "w") as md_file:
    md_file.write(
        tabulate(
            columns, headers=headers, showindex=False, tablefmt="github", intfmt=","
        )
    )

Title,Number of rows,Download data,View at State Archives
Aboriginal People in the Register of Aboriginal Reserves 1875-1904,78,CSV file,Browse index
Assisted Immigrants Index 1839-1896,"200,000",CSV file,Browse index
Australian Railway Supply Detachment 1914,65,CSV file,Browse index
Bankruptcy index 1888-1929,"30,000",CSV file,Browse index
Bench of Magistrates Index 1788-1820,"4,442",CSV file,Browse index
Botanic Gardens and government domains employees,916,CSV file,Browse index
Bubonic plague index 1900-1908,567,CSV file,Browse index
Census - 1841,"9,355",CSV file,Browse index
"Chemists, druggists and pharmacists index 1876-1920","2,967",CSV file,Browse index
Child care and protection index 1817-1942,"21,292",CSV file,Browse index


----

Created by [Tim Sherratt](https://timsherratt.org/) for the [GLAM Workbench](https://glam-workbench.net/) project.